# Local Agentic PostgreSQL Bundle (Postgres 16) — Notebook

This notebook guides you through launching a local Postgres 16 instance via Docker Compose, populating it with sample e-commerce and observability data, and running simple integration tests.

Files created in `/mnt/data/agentic_local_bundle`:

- `docker-compose.yml` — Postgres 16 service
- `initdb/init.sql` — schema + extensions
- `generate_data.py` — Python script to populate datasets
- `tests/test_agent.py` — pytest harness

Run the commands in the following cells in your local environment (they will not run in this sandbox).


## 1) Start Postgres with Docker Compose

From the directory containing `docker-compose.yml` run:

```bash
docker compose up -d
```

This will expose Postgres on `localhost:5432` with credentials `agentic/agentic_pass` and database `agentic_db`.


In [ ]:
# Optional: bring up Docker Compose (DO NOT RUN in this sandbox)
!docker compose -f /mnt/data/agentic_local_bundle/docker-compose.yml up -d
print('docker compose up requested')


## 2) Wait for DB and generate dataset

Run the Python data generator to populate the database. By default it seeds 50 customers, 40 products, 200 orders, 1000 metrics, 50 alerts, 500 logs.

```bash
python /mnt/data/agentic_local_bundle/generate_data.py --customers 50 --products 40 --orders 200 --metrics 1000 --alerts 50 --logs 500
```


In [ ]:
# Optional: run generator from the notebook (DO NOT RUN here)
!python /mnt/data/agentic_local_bundle/generate_data.py --customers 50 --products 40 --orders 200 --metrics 1000 --alerts 50 --logs 500
print('data generation requested')


## 3) Quick sanity-check queries

Use SQLAlchemy to connect and run example queries against commerce and observability schemas.


In [ ]:
# Example queries (DO NOT RUN in this sandbox)
from sqlalchemy import create_engine, text
import os

PG_USER = os.getenv('PGUSER','agentic')
PG_PASS = os.getenv('PGPASSWORD','agentic_pass')
PG_HOST = os.getenv('PGHOST','localhost')
PG_PORT = os.getenv('PGPORT','5432')
PG_DB   = os.getenv('PGDB','agentic_db')

engine = create_engine(f'postgresql+psycopg2://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}')
with engine.connect() as conn:
    q1 = text("SELECT p.name, SUM(oi.quantity*oi.unit_price) as revenue FROM commerce.order_items oi JOIN commerce.products p ON oi.product_id = p.id GROUP BY p.name ORDER BY revenue DESC LIMIT 5")
    res = conn.execute(q1).fetchall()
    print('Top products by revenue:', res)

    q2 = text("SELECT alert_name, COUNT(*) as c FROM observability.alerts GROUP BY alert_name ORDER BY c DESC LIMIT 5")
    res2 = conn.execute(q2).fetchall()
    print('Top alerts:', res2)


## 4) Run tests

From the bundle directory run:

```bash
pip install -r requirements.txt  # see requirements below
pytest -q /mnt/data/agentic_local_bundle/tests
```

Expected: tests pass if DB is up and data generated.


## 5) Requirements

Install these locally in your virtualenv:

```bash
pip install sqlalchemy psycopg2-binary psycopg2 pytest psycopg2-binary
```


## 6) Next: LangChain integration

Once the DB is up and seeded, continue with the LangChain notebook (or the earlier notebook scaffolds) to connect the agent in read-only mode and run example prompts.
